#### Задача расчёта матрицы корреспонденций

$\sum_{i, j = 1, 1}^{n, n}{T_{i, j}x_{i, j} + \gamma \sum_{i, j = 1, 1}^{n, n}x_{i, j}\log{x_{i, j}}} \rightarrow \min$

$\sum_{j = 1}^n {x_{i, j} = l_i}, $
$\sum_{i = 1}^n {x_{i, j} = W_j}, $
$ x_{i,j} \ge 0
$

$\langle{\lambda, l}\rangle + \langle{\mu, W}\rangle + \gamma \log{\sum_{i, j}^{n, n} 
\exp{\frac{\lambda_i + \mu_j - T_{}i, j}{\gamma}}} \rightarrow \min_{\lambda, \mu} $

Это двойственная задача к указанной выше, при условии $\sum_{i=1, j=1}^{n, n} x_{i, j} = 1$. Как её решать? Можно с помощью быстрого градиентного спуска (см ниже).

$x^{k + 1} = x^k - \frac{1}{L} \nabla f(x^k + \frac{\sqrt{L} - \sqrt{\mu}}{\sqrt{L} + \sqrt{\mu}}(x^k - x^{k - 1})) + \frac{\sqrt{L} - \sqrt{\mu}}{\sqrt{L} + \sqrt{\mu}}(x^k - x^{k - 1})$

1. Выбираем $\mu$. 
2. Потом $\mu = \frac{\mu}{2}$.
3. И так далее
4. Если сошлось с \mu сильно больше нуля (как я поняла), то мы пришли к успеху. Ищем невязку между двойственной ф-ей и исходной (как? спросить)

In [1]:
import numpy as np
import pandas as pd
# np.set_printoptions(suppress=True) # чтоб без e
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import axes3d, Axes3D
import time
from pylab import *
from math import exp, log, sqrt

%matplotlib inline

In [2]:
start_time = time.time()
df = pd.read_csv('traffic_moscow.csv', header=None)
df.columns = ["i", "j", "amt", "t", "s"]
print("--- %s seconds ---" % (time.time() - start_time))
df.head()

--- 0.020645856857299805 seconds ---


,i,j,amt,t,s
0,1,1,40,26,2.2
1,1,2,4,49,8.1
2,1,3,3,35,6.8
3,1,4,2,75,9.3
4,1,5,2,30,8.0


In [3]:
def make_index(i, j):
    residents = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
    if int(i) in residents:
        return 1 # живут в Москве
    else:
        return 0 # живут в области

In [4]:
df['r'] = df.apply(lambda row: make_index(row['i'], 
                                              row['j']), axis=1) # добавили столбец

In [5]:
def amt_for_residents(df): # умножили значения москвичей на 1.62
    for i in range(df.shape[0]):
        df.loc[i, 'amt'] *= 1.62 if df.loc[i, 'r'] == 1 else 1
    return df

df = amt_for_residents(df)

In [6]:
L_i = df.groupby('i')['amt'].sum().reset_index()
W_j = df.groupby('j')['amt'].sum().reset_index()

total_i = L_i['amt'].sum()
total_j = W_j['amt'].sum()

print(total_i, total_j)

2571.36 2571.3599999999997


In [7]:
print(np.shape(W_j))

(22, 2)


Подготовили данные. Умножили значения москвичей на 1.62, т.к. их больше. $W_j$ и $L_i$ выкачали из датафрейма

__ВОПРОС:__ По условию задачи $\sum_{i, j}^{i, j}{x_{i, j}} = 1$. У нас 2571,36. Отнормировать?

__ОТВЕТ__ Да. А перед тем как считать невязку -- вернуть как было

In [8]:
def find_Lipschitz_constant(gamma):
    return 1 / gamma
    
def find_gamma(df):
    n = 21
    t = 0
    for i in range(n):
        for j in range(n):
            t += costs_func(i, j)
    gamma = t/n
    return gamma

def costs_func(i, j): # возвращаем время T_i_j
    rows = df.loc[df['i'] == i]
    columns = rows.loc[df['j'] == j]
    T = columns['t'] # потому что издержки -- это время
    try:
        return int(T)
    except TypeError: # если не пересекается район i с районом j
        return 0

def dual_f(df, lmbd, zeta, L_i, W_j):
    gamma = find_gamma(df)
    n = 21 # число районов
    value = 0
    for i in range(n):
        for j in range(n):
            T = costs_func(i, j)
            value += exp((lmbd[i] + zeta[j] - T)/gamma)
    return np.dot(lmbd, L_i) + np.dot(zeta, W_j) + gamma * math.log(value)

__ВОПРОС:__ Как искать липшицеву константу $M_1$ (L в лекции). В работе Мерузы сначала L=1, потом L=L/2. Так делать?

__ОТВЕТ:__ $L = 1/\gamma$, а $\gamma = $ среднее арифметическое всех $T_{i, j}$

__ВОПРОС:__ Забыла записать как считается гамма, спросить и ЗАПИСАТЬ
__ОТВЕТ:__ $\gamma = $ среднее арифметическое всех $T_{i, j}$

__ВОПРОС:__ мю и лямбда векторы или не векторы.
1. Если векторы, как считается корень из мю в градиенте в FGM
2. Если не векторы, скалярное произведение ли в двойственной задаче?

__ОТВЕТ:__ Неудачное обозначение.
1. $\mu$ и $\lambda$ из двойственной функции и из выражения для градиентного спуска -- это разные вещи.
2. $\mu$ и $\lambda$ из двойственной функции равны:

$(\lambda_1, ..., \lambda_n) = (x_1, ...., x_n)$

$(\mu_1, ..., \mu_n) = (x_{n+1}, ...., x_{2n})$
3. $\mu$ из двойственной функции ищется как сначала $\mu = 1$, потом $\mu$ = $\mu / 2$ и т.д. Как ищется липшицева константа L написано выше

In [9]:
def find_grad(M_1, mu, x, x_prev, lmbd, gamma, L_i, W_j):
    v =  x + ((sqrt(M_1) - sqrt(mu))/(sqrt(M_1) + sqrt(mu)))*(x - x_prev)
    f_v = dual_f(lmbd=v, mu=mu, gamma=gamma, L_i=L_i, W_j=W_j)
    return np.gradient(f_v)

def FGM(x, x_prev, M_1, mu):
    grad_f = find_grad(L, mu, x, x_prev)
    x_next = x - 1/M_1* grad_f + ((sqrt(M_1) - sqrt(mu))/(sqrt(M_1) + sqrt(mu)))*(x - x_prev)
    return x_next

In [ ]:
n = L_i.shape[0]
lmbd = np.ones(n)
zeta = np.ones(n)

gamma = find_gamma(df)
M_1 = find_Lipschitz_constant(gamma)
x_next = np.ones(2*n)
x_prev = np.ones(2*n)
x = np.ones(2*n)
mu = 1.0
mu_list = [mu]
x_list = [x_prev, x, x_next]

for i in range(2*n):
    x_next = FGM(x, x_prev, M_1, mu)
    x_prev = x
    x = x_next
    mu = mu/2
    mu_list.append(mu)
    x_list.append(x_next)